# Algumas funções para ajuste de modelos de regressão

### Bibliotecas

In [ ]:
if(!require(car)) install.packages("car", repos = "http://cran.us.r-project.org")
if(!require(tidyverse)) install.packages("dplyr", repos = "http://cran.us.r-project.org")

### Conjunto de dados

In [ ]:
help(package="carData")

### Alguns comandos úteis

In [ ]:
class(Duncan)  # Dados de prestigio ocupacional de Duncan (1961), "A socioeconomic index for all occupations"

In [ ]:
Duncan.tibble <- as_tibble(Duncan); class(Duncan.tibble)

In [ ]:
head(Duncan.tibble)

In [ ]:
str(Duncan)

In [ ]:
search()

In [ ]:
Duncan$prestige
mean(Duncan$prestige)
mean(Duncan[ , "prestige"]) # equivalent, column by name
mean(Duncan[ , 4]) # equivalent, column by number

### Ajustando um modelo de regressão

In [ ]:
fit <- lm(prestige ~ income + education, data=Duncan)

In [ ]:
summary(fit)

In [ ]:
plot(fit)

In [ ]:
set.seed(1234)
some(Duncan$prestige) # Use a função do pacote car

In [ ]:
with(Duncan, hist(income))
box()

In [ ]:
densityPlot(rstudent(fit)) # Gráfico de resíduos studentizados

In [ ]:
influence.measures(fit)

In [ ]:
influenceIndexPlot (fit, vars=c ("Cook", "hat"), id=list(n=3))

In [ ]:
outlierTest(fit)

In [ ]:
qqPlot(~ income, data=Duncan, id=list(n=5))

### Alguns testes de normalidade

In [ ]:
normality.test<-function(dados){
    if(!require(nortest)) install.packages("nortest", repos = "http://cran.us.r-project.org")
    t1 <- ks.test(dados, "pnorm", mean(dados), sd(dados))  # KS
    t2 <- lillie.test(dados)  # Lilliefors
    t3 <- cvm.test(dados)  # Cramér-von Mises
    t4 <- shapiro.test(dados) # Shapiro-Wilk
    t5 <- sf.test(dados) # Shapiro-Francia
    t6 <- ad.test(dados) # Anderson-Darling
    # Tabela de resultados
    testes <- c(t1$method, t2$method, t3$method, t4$method, t5$method,t6$method)
    estatistica <- as.numeric(c(t1$statistic, t2$statistic, t3$statistic,t4$statistic, t5$statistic, t6$statistic))
    valor_p <- c(t1$p.value, t2$p.value, t3$p.value, t4$p.value, t5$p.value,t6$p.value)
    resultados <- cbind(estatistica, valor_p)
    rownames(resultados) <- testes
    colnames(resultados) <- c("Estatística", "Valor p")
    print(resultados, digits = 4)
}

normality.test(fit$res)

### Pacote PoweR

Documentação do pacote [PoweR](https://cran.r-project.org/web/packages/PoweR/PoweR.pdf)

In [ ]:
if(!require(PoweR)) install.packages("PoweR", repos = "http://cran.us.r-project.org")

statcompute(21, fit$res) # Shapiro-Wilks

In [ ]:
Boxplot(~ income, data=Duncan)

In [ ]:
with(Duncan, plot(income, prestige))

In [ ]:
scatterplot(prestige ~ income, data=Duncan, id=list(n=4))

In [ ]:
Prestige$type <- factor(Prestige$type,
    levels=c("bc", "wc", "prof"))

In [ ]:
scatterplot(prestige ~ income | type, data=Duncan,
    legend=list(coords="bottomright", inset=0.1),
    smooth=list(span=0.9))

In [ ]:
xtabs(~type, data=Duncan)

In [ ]:
scatterplotMatrix(~ prestige + income + education + women,data=Prestige)

In [ ]:
scatterplot(infantMortality ~ ppgdp, data=UN,
     xlab="GDP per Capita",
     ylab="Infant Mortality Rate (per 1000 births)",
     main="(a)", id=list(n=3))

In [ ]:
scatterplot(infantMortality ~ ppgdp, data=UN,
     xlab="GDP per capita",
     ylab="Infant Mortality Rate (per 1000 births)",
     main="(b)", log="xy", id=list(n=3))

In [ ]:
fit<-lm(log(infantMortality) ~ log(ppgdp), data=UN)
summary(fit)

In [ ]:
S(fit)  # Estimativas dos coeficientes

In [ ]:
Confint(fit) # INtervalos de confiança para os coeficientes

### Transformações para normalidade

In [ ]:
summary(p1 <- powerTransform(infantMortality ~ 1, data=UN, family="bcPower"))

In [ ]:
testTransform(p1, lambda=1/2)

In [ ]:
summary(p2 <- powerTransform(cbind(income, education) ~ 1, data=Prestige, family="bcPower"))

### Comparação de modelos: caso de remoção de pontos influentes

In [ ]:
davis.mod <- lm(weight ~ repwt, data=Davis)
plot(weight ~ repwt, data=Davis)
abline(0, 1, lty="dashed", lwd=2)
abline(davis.mod, lwd=2)
legend("bottomright", c("Unbiased Reporting", "Least Squares"),lty=c("dashed", "solid"), lwd=2, inset=0.02)
with(Davis, showLabels(repwt, weight, n=2, method="mahal"))

In [ ]:
davis.mod.2 <- update(davis.mod, subset=-12)
S(davis.mod.2)

In [ ]:
compareCoefs(davis.mod, davis.mod.2)
Confint(davis.mod.2)

In [ ]:
carWeb() # Pegar os arquivos na Web